In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Texts
- Embedding
- ChromaDB
- QA Chain
- Source info

## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_groq import ChatGroq
from glob import glob
import os


## Models to use

In [4]:
embedding_name = 'nomic-ai/nomic-embed-text-v1.5'

In [5]:
# Load embedding
model_kwargs = {'trust_remote_code': True}
embedding = HuggingFaceEmbeddings(model_name=embedding_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [6]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

# Initializing GROQ chat with provided API key, model name, and settings
model = ChatGroq(
            groq_api_key=groq_api_key, model_name="llama3-70b-8192",
                         temperature=0.2)

## Load multiple and process documents

In [7]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

In [8]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [9]:
len(texts)

159

In [10]:
for text in texts:
  print(text.metadata)

{'source': '01-Description.txt'}
{'source': '01-Description.txt'}
{'source': '01-Description.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': '03-The University Seal.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '04-History.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '05-University Saints.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '06-USC Organizational Structure.txt'}
{'source': '07-Admission.txt'}
{'source': '07-Admission.txt'}
{'source': '08-Admission Steps for Freshmen.txt'}
{'source': '08-Admission Steps for Freshmen.txt'}
{'source': '09-Admission Steps for International Students.txt'}

## Create the DB

In [11]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [12]:
# persiste the db to disk
vectordb.persist()
vectordb = None

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [13]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [14]:
retriever = vectordb.as_retriever()

In [15]:
docs = retriever.get_relevant_documents("What are the saints of University of San Carlos?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
len(docs)

4

In [17]:
for doc in docs:
  print(doc.metadata)

{'source': '04-History.txt'}
{'source': '05-University Saints.txt'}
{'source': '04-History.txt'}
{'source': '60-Directory of Student Support Services and Programs.txt'}


## Make a chain

In [21]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=model,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [22]:
## Cite sources
def process_llm_response(llm_response):
    if('According to the provided context, ' in llm_response['result']):
      llm_response['result'] = llm_response['result'][35:]
      llm_response['result'] = llm_response['result'][0].upper() + llm_response['result'][1:]
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [23]:
# full example
query = "What is the passing grade?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

A passing grade is a grade within 1.0 and 3.0.


Sources:
37-Academic Performance and Grading System.txt
37-Academic Performance and Grading System.txt
39-Graduation.txt
35-Curricular and Non-Curricular Requirements.txt


In [24]:
llm_response

{'query': 'What is the passing grade?',
 'result': 'A passing grade is a grade within 1.0 and 3.0.',
 'source_documents': [Document(page_content='A grade of 1.0 is equal to letter grade A+. A grade of 1.1 to 1.3 is equal to letter grade A. A grade of 1.4 to 1.7 is equal to letter grade A-. A grade of 1.7 to 1.9 is equal to letter grade B+. A grade of 2.0 to 2.2 is equal to letter grade B. A grade of 2.3 to 2.5 is equal to letter grade B-. A grade of 2.6-2.8 is equal to letter grade C+. A grade of 2.9 to 3.0 is equal to letter grade C. A grade of 5.0 is equal to C-.\n\nA passing grade is a grade within 1.0 and 3.0. A grade of 5.0 means failure.\n\nUnsatisfactory performance or unexcused absences of more than 20% of the prescribed number of class hours or laboratory periods during the term merits a 5.0 or failure as well.\n\nYou need a numeric grade within 1.0 to 3.0 to pass a given course. Take note that it is based solely on the final grade, while the midterm grades are completely irre

In [25]:
# another example
query = "What is block enrollment?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


Block enrollment is a type of enrollment that allows a student to enroll all the courses for the term at once. To be eligible for block enrollment, a student must be enrolling for a regular full load in a given semester, must not have incurred any failures or NG from previous semesters, and must have obtained passing grades for all courses that are pre-requisite to the set of courses on a given semester.


Sources:
17-Enrollment Timeline.txt
17-Enrollment Timeline.txt
18-Enrollment Steps for Continuing Students.txt
18-Enrollment Steps for Continuing Students.txt


In [26]:
llm_response

{'query': 'What is block enrollment?',
 'result': 'Block enrollment is a type of enrollment that allows a student to enroll all the courses for the term at once. To be eligible for block enrollment, a student must be enrolling for a regular full load in a given semester, must not have incurred any failures or NG from previous semesters, and must have obtained passing grades for all courses that are pre-requisite to the set of courses on a given semester.',
 'source_documents': [Document(page_content='Non-block sectioning requires the student to advise and enroll a course one at a time. This is available to all students regardless of their course statuses and only after the block enrollment period has ended. During non-block enrollment, the course schedules which the students can view during this period are only those which are reserved to their home department.', metadata={'source': '17-Enrollment Timeline.txt'}),
  Document(page_content='Topic 17: Enrollment Timeline\n\nThe order of e

In [27]:
# another example
query = "How do I shift to another program?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)


To shift to another program, you need to follow these steps:

1. Click "Student Task", "Others", and "Apply Change of Program".
2. Select the option that applies to you between "Change Program (Same Department)" or "Change Program (Different Department)".
3. Indicate the academic period, year, school, department, and program where you want to shift to.
4. Place a reason for shifting to that program and then click "Submit".
5. Monitor the status at "Application History" which includes the old chair for endorsement, Counseling and Development Center for interview, the new chair for endorsement, and registrar for approval.

Once approved, the Registrar staff will tag you to your new program. Then, proceed to non-block sectioning enrollment, and afterwards, surrender your old RFID to OSFA office, and pay 250 pesos for reprinting and proceed to ID room and bring your blue slip from OSFA and OR for your new RFID.


Sources:
28-Additional Enrollment Steps (Shiftee).txt
29-Additional Enrollmen

In [28]:
# another example
query = "What is the difference between overload and tutorial?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

Based on the provided context, overload and tutorial are two different concepts:

Overload refers to a special case where a graduating student can enroll in more than the maximum allowed units in a semester, up to a maximum of 6 units, to graduate on time. This is subject to certain conditions, such as a GPA of at least 2.8 and not having more than 5 failures.

Tutorial, on the other hand, is a type of course offering that can be requested by students as a petitioned or tutorial class. This is typically for off-semester courses, and the type of offering (tutorial or petition) depends on the number of students signing up. Tutorials are usually for small groups of students (1-5) and may incur additional charges.

In summary, overload is about taking extra units in a semester, while tutorial is a type of course offering for off-semester courses with a limited number of students.


Sources:
23-Overload.txt
35-Curricular and Non-Curricular Requirements.txt
25-Tutorial and Petition.txt
21-Co

In [32]:
# another example
query = "What type of offense is hazing?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

Hazing is a Very Serious Offense (D) as stated in Offenses against security, safety, and order, item 12.


Sources:
51-List of Offenses.txt
50-Categories and Ranking of Offenses - Copy.txt
51-List of Offenses.txt
51-List of Offenses.txt


In [33]:
# another example
query = "What grades equal to percentages above 90%?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

Based on the provided context, a grade of 1.0 is equal to a letter grade of A+, which corresponds to a percentage above 90%.


Sources:
37-Academic Performance and Grading System.txt
35-Curricular and Non-Curricular Requirements.txt
37-Academic Performance and Grading System.txt
35-Curricular and Non-Curricular Requirements.txt


In [34]:
# another example
query = "What happens if you commit a serious offense"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

If you commit a serious offense, the following steps will be taken:

1. All teachers, school officials, and other university employees and persons in authority may call the attention of the student who committed a serious offense.
2. The case will be handled by the campus student discipline office (SDO).
3. After an initial assessment and evaluation, the campus SDO may recommend for the Student Development Mentors (SDMs) and/or Student Development Committee (SDEVCOM) to handle the case or endorse it, and then convene the Student Discipline Committee.
4. The student will have a Discipline Conversation (Written Reply). The chair, the dean, and the Parents/Guardians will be informed.
5. The Student Discipline Committee will be convened, and the student will receive a Written Warning and will be required to submit a written reply.

Note that serious offenses are recorded in a student's name during their stay in the University and may prevent them from receiving a clean Certificate of Good 

In [36]:
# another example
query = "How do I access my USC email?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

To access your USC email, you can use your USC Google Workspace account. Your email address is in the format "idnumber@usc.edu.ph" (e.g. 12345@usc.edu.ph). Your default password is your lastname + your idnumber (e.g. delacruz12345). Once logged in, you are advised to set up a recovery email so that you can reset your password anytime in case you forget.


Sources:
33-USC Email and Canvas.txt
60-Directory of Student Support Services and Programs.txt
15-Post-Admission Enrollment Steps.txt
32-Transferring out of USC.txt


In [37]:
# another example
query = "How do I enroll?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

The enrollment steps for continuing students are:

1. Log in to your ISMIS account.
2. Go to the student enrollment function by clicking "Student Task", "Enrollment Related", and "Student Enrollment".
3. Settle your accountabilities, if any.
4. For block enrollment, advise and enroll for a block section by clicking "Block Advising", choosing your desired Block Section, and then clicking "Enroll Block Section".
5. For non-block enrollment, advise and enroll courses one by one by clicking "Advise Course", selecting from the available schedules, and then clicking "Enroll Course".
6. Enroll in GE Free Electives by following the steps outlined in Step 5.
7. View your study load by clicking "Student Task", "Enrollment Related", and "View Study Load".

Note that these steps are specific to continuing students, and the enrollment process may differ for new students.


Sources:
18-Enrollment Steps for Continuing Students.txt
18-Enrollment Steps for Continuing Students.txt
16-Enrollment Mechanic

In [38]:
# another example
query = "I am top 3 in my CHL cluster. What discounts can I get?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

According to the context, as a top 3 student in your CHL cluster, you can get a tuition fee discount scholarship. The discount rates are:

* 1st in rank: 100% tuition fee discount
* 2nd in rank: 75% tuition fee discount
* 3rd in rank: 50% tuition fee discount


Sources:
37-Academic Performance and Grading System.txt
41-Recognition and Previleges.txt
42-Qualification and percentage distribution.txt
31-Fee Discounts.txt
